In [1]:
import folium
import pandas as pd

print(folium.__version__)

0.10.1


In [2]:
# define the world map
world_map = folium.Map()

# display world map
world_map

In [3]:
# beijing latitude and longitude values
cen_latitude = 39.93
cen_longitude = 116.40

# Create map and display it
bj_map = folium.Map(location=[cen_latitude, cen_longitude], zoom_start=12)

# Display the map of beijing
bj_map

In [4]:
# Change tiles of map
bj_map = folium.Map(location=[cen_latitude, cen_longitude], zoom_start=12, tiles='Stamen Toner')
bj_map

In [5]:
# Read Dataset 
#cdata = pd.read_csv('https://cocl.us/sanfran_crime_dataset')
#cdata.head()
data = pd.read_csv("bj_res.txt",sep='\t')
data.lng

0       116.411455
1       116.418797
2       116.414432
3       116.422525
4       116.427536
           ...    
1995    116.502089
1996    116.501532
1997    116.482461
1998    116.484569
1999    116.490339
Name: lng, Length: 2000, dtype: float64

In [6]:
print(data)
incidents = folium.map.FeatureGroup()

# Loop data to the incidents feature group
for lat, lng, in zip(data.lat, data.lng):
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=7, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='red',
            fill_opacity=0.4
        )
    )

# Add incidents to map
bj_map = folium.Map(location=[cen_latitude, cen_longitude], zoom_start=12)
bj_map.add_child(incidents)

            lat         lng district_name
0     39.936250  116.411455           东城区
1     39.906203  116.418797           东城区
2     39.977111  116.414432           东城区
3     39.923523  116.422525           东城区
4     39.943605  116.427536           东城区
...         ...         ...           ...
1995  39.806041  116.502089         亦庄开发区
1996  39.805667  116.501532         亦庄开发区
1997  39.806520  116.482461         亦庄开发区
1998  39.802959  116.484569         亦庄开发区
1999  39.804621  116.490339         亦庄开发区

[2000 rows x 3 columns]


In [7]:
# add pop-up text to each marker on the map
import numpy as np
bj_map = folium.Map(location=[cen_latitude, cen_longitude], zoom_start=11)
latitudes = list(data.lat)
longitudes = list(data.lng)
labels = np.random.randint(50000,100000,size = len(data))

for lat, lng, label in zip(latitudes[:10], longitudes[:10], labels):
    folium.Marker([lat, lng], radius=30,popup=folium.map.Popup("<font color=red>hhhhhh测试中文</font>",show=True,sticky=True)).add_to(bj_map)  
bj_map

In [8]:
from folium import plugins

# let's start again with a clean copy of the map of beijing
bj_map = folium.Map(location = [cen_latitude, cen_longitude], zoom_start = 12)

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(bj_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(data.lat, data.lng, labels):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

# add incidents to map
bj_map.add_child(incidents)

In [9]:
import json
import requests

with open("bj_geo.json") as f:
    bj_geo = json.load(f)

bj_map = folium.Map(location=[cen_latitude, cen_longitude], zoom_start=12)
folium.GeoJson(
    bj_geo,
    style_function=lambda feature: {
        'fillColor': '#ffff00',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(bj_map)

#display map
bj_map

In [10]:
# Count crime numbers in each neighborhood
disdata = pd.DataFrame(data['district_name'].value_counts())
disdata.reset_index(inplace=True)
disdata.rename(columns={'index':'DistrictName','district_name':'Count'},inplace=True)
disdata

,DistrictName,Count
0,朝阳区,466
1,西城区,367
2,海淀区,332
3,东城区,315
4,丰台区,204
5,通州区,82
6,昌平区,64
7,大兴区,51
8,石景山区,44
9,房山区,28


In [11]:
bj_map = folium.Map(location=[cen_latitude, cen_longitude], zoom_start=11)

# Create Choropleth map
folium.Choropleth(
    geo_data=bj_geo,
    data=disdata,
    columns=['DistrictName','Count'],
    key_on='feature.properties.DISTRICT',
    #fill_color='red',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    highlight=True,
    legend_name='Resblock Counts in Beijing'
).add_to(bj_map)

bj_map.save("test.html")
bj_map

In [12]:
from folium.plugins import HeatMap

# let's start again with a clean copy of the map of beijing
bj_map = folium.Map(location = [cen_latitude, cen_longitude], zoom_start = 12)

# Convert data format
heatdata = data[['lat','lng']].values.tolist()

# add incidents to map
HeatMap(heatdata).add_to(bj_map)

bj_map

In [13]:
import geopandas as gpd
import branca, fiona
disdata

states = gpd.GeoDataFrame.from_features(bj_geo,crs=fiona.crs.from_epsg(4326))
states.head()

statesmerge = states.merge(disdata,how="left",left_on="DISTRICT",right_on="DistrictName")
statesmerge.head()



,geometry,OBJECTID,DISTRICT,COMPANY,name,DistrictName,Count
0,"POLYGON ((116.42041 39.86637, 116.42036 39.870...",1,东城区,A,东城区,东城区,315.0
1,"POLYGON ((116.33966 39.94376, 116.34111 39.928...",1,西城区,A,西城区,西城区,367.0
2,"POLYGON ((116.39883 39.96959, 116.39590 39.969...",1,朝阳区,A,朝阳区,朝阳区,466.0
3,"POLYGON ((116.10054 39.83431, 116.10009 39.819...",1,丰台区,A,丰台区,丰台区,204.0
4,"POLYGON ((116.25950 39.90305, 116.25932 39.927...",1,石景山区,A,石景山区,石景山区,44.0


In [14]:
# 定义颜色渐变
colormap = branca.colormap.LinearColormap(vmin=statesmerge['Count'].quantile(0.0), 
                                        vmax=statesmerge['Count'].quantile(1), 
                                        colors=['darkgreen','green','lightblue','orange','red'],
                                       caption="Number of Resblock of District in Beijing")
# 创建地图
bj_map = folium.Map(location=[cen_latitude, cen_longitude], zoom_start=11)

# 鼠标停留的显示配置
tooltip = folium.features.GeoJsonTooltip(
    fields=["DistrictName", "Count"],
    aliases=["DistrictName:", "Count:"],
    localize=True,
    sticky=True,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 1.5px solid black;
        border-radius: 3px;
        box-shadow: 1px;
    """)

g = folium.GeoJson(
    statesmerge,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["Count"])
        if x["properties"]["Count"] is not None
        else "transparent",
        "color": "black",
        "fillOpacity": 0.4,
    },
    highlight_function=lambda x: {
        "fillColor": colormap(x["properties"]["Count"])
        if x["properties"]["Count"] is not None
        else "transparent",
        "color": "black",
        "fillOpacity": 0.7,
    },
    tooltip=tooltip,
).add_to(bj_map)
colormap.add_to(bj_map)

bj_map.save("bj_show.html")
bj_map
